In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn import datasets, ensemble
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import datetime as dt
from sklearn import  metrics 
from sklearn.metrics import  make_scorer 
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 
import lightgbm as lgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import time
import sys
from catboost import CatBoostClassifier

In [ ]:
train= pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test= pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')


In [ ]:
train['Response'].value_counts()


In [ ]:

train.head()


In [ ]:
bins = [18, 30, 40, 50, 60, 70, 80, 90, 100]
lab = ['2', '3', '4', '5', '6', '7','8','9']
train['agerange'] = pd.cut(train['Age'], bins, labels = lab,include_lowest = True)
test['agerange'] = pd.cut(test['Age'], bins, labels = lab,include_lowest = True)

train['agerange']=train['agerange'].apply(np.int64)
test['agerange']=test['agerange'].apply(np.int64)



In [ ]:
train['agerange']=train['agerange'].astype(int)
test['agerange']=test['agerange'].astype(int)

In [ ]:
train['Vehicle_Damage']=train['Vehicle_Damage'].map({'Yes': 10,'No': 1}).astype(int)
test['Vehicle_Damage']=test['Vehicle_Damage'].map({'Yes': 10,'No': 1}).astype(int)

train['Vehicle_Age']=train['Vehicle_Age'].map({'< 1 Year' : 0,'1-2 Year': 1,'> 2 Years': 2}).astype(int)
test['Vehicle_Age']=test['Vehicle_Age'].map({'< 1 Year' : 0,'1-2 Year': 1,'> 2 Years': 2}).astype(int)

train['Vehicle_Age'].apply(np.int64)
test['Vehicle_Age'].apply(np.int64)

train['Vehicle_Damage'].apply(np.int64)
test['Vehicle_Damage'].apply(np.int64)

train['Vehicle_Damage_Age']= train['Vehicle_Damage']*train['Vehicle_Age']
test['Vehicle_Damage_Age']= test['Vehicle_Damage']*test['Vehicle_Age']

train['mean_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('mean')
test['mean_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('mean')

train['max_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('max')
test['max_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('max')

train['sum_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('sum')
test['sum_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('sum')


train['count_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['id'].transform('count')
test['count_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['id'].transform('count')

train['min_annual_premium_policy_sales_channel']=train.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('min')
test['min_annual_premium_policy_sales_channel']=test.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('min')

train['Annual_Premium_Vintage']=train['Annual_Premium']* train['Vintage']
test['Annual_Premium_Vintage']=test['Annual_Premium']* test['Vintage']



In [ ]:
train['Gender']=train['Gender'].map({'Male' : 0,'Female': 1}).astype(int)
test['Gender']=test['Gender'].map({'Male' : 0,'Female': 1}).astype(int)


In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

c= np.unique(train['Response'])

weigh = compute_class_weight(class_weight='balanced',classes= c,  y=train['Response'])
#weigh= pd.Series(weigh)

In [ ]:
#46710

In [ ]:
train[train['Response']==0].count()

In [ ]:
y=train['Response']

In [ ]:
predictors = [x for x in train.columns if x not in ['Response', 'id']]

XGBOOST PARAM

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=None)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], y,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y, dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
predictors = [x for x in train.columns if x not in ['Response', 'id']]
xgb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=4, min_child_weight=4, 
                     gamma=0.45, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', 
                     nthread=-1, seed=27, scale_pos_weight= 7.16)
modelfit(xgb1, train, predictors)

In [ ]:
print("Best Iteration: {}".format(xgb1.get_booster().best_iteration))

In [ ]:
param_test1 = {
    'reg_lambda': [25,30]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=243, max_depth=4,
                                                  min_child_weight=4, gamma=0.45, subsample=0.8, colsample_bytree=0.8, reg_alpha= 0.01,
                                                  reg_lambda= 25,
                                                  objective= 'binary:logistic', nthread=4, scale_pos_weight=7.16, seed=27), 
                        param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(train[predictors],y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
mod= XGBClassifier( learning_rate =0.01, n_estimators=5000, max_depth=4,
                   min_child_weight=4, gamma=0.45, subsample=0.8, colsample_bytree=0.8, reg_alpha= 0.01,
                   reg_lambda= 25,objective= 'binary:logistic', nthread=4, scale_pos_weight=7.16, seed=27)
mod.fit(train[predictors],y)
preds= mod.predict_proba(test[predictors])[:, 1]

In [ ]:
pre= pd.Series(preds)
pre.value_counts()

In [ ]:
tid=test['id']

In [ ]:
sub = pd.concat([tid, pre], axis=1)
sub

In [ ]:
sub.rename({0: 'Response'},axis=1, inplace= True)

In [ ]:
sub.to_csv('submissionY.csv', index=False)

CATBOOST PARAM

In [ ]:
params = {#'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          #'l2_leaf_reg':[3,1,5,10,100],
          #'border_count':[32,5,10,20,50,100,200],
          #'ctr_border_count':[50,5,10,20,100,200],
          #'thread_count':-1
}

In [ ]:
categorical_features = ['Gender', 'Driving_License', 'Previously_Insured', 'Vehicle_Age','agerange',
                        'Region_Code', 'Policy_Sales_Channel','Vehicle_Damage', 
                        'count_annual_premium_policy_sales_channel',
                        #'count_annual_premium_region_code'
                       ]

In [ ]:
train['count_annual_premium_policy_sales_channel'].value_counts()

In [ ]:

train['Region_Code']=train['Region_Code'].apply(np.int64)
test['Region_Code']=test['Region_Code'].apply(np.int64)

train['Policy_Sales_Channel']=train['Policy_Sales_Channel'].apply(np.int64)
test['Policy_Sales_Channel']=test['Policy_Sales_Channel'].apply(np.int64)

In [ ]:
x.columns

In [ ]:
y = x['Response']
x.drop(['id','Response'],axis=1, inplace= True)
#test.drop(['id','Response'],axis=1, inplace= True)
test_case_id= test['id']
test.drop(['id'],axis=1, inplace= True)

In [ ]:
train, test, y_train, y_test = train_test_split(train, y,
                                                random_state=10, test_size=0.25,
                                                stratify=y)

In [ ]:
train.columns

In [ ]:
import catboost as cb
#cat_features_index = [0,1,2,3,4,5,6]

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

params = {
    'n_estimators' : [250],
    'depth': [6],
    'objective' : ['Logloss'],
    'learning_rate' : [0.1],
    'l2_leaf_reg': [9],
    'bootstrap_type' : ['Bayesian'],
    'colsample_bylevel' : [0.9],
    'bagging_temperature' : [1],
    'border_count':[50],
    'num_leaves' : []
    #'iterations': [300]
}
cb = cb.CatBoostClassifier(scale_pos_weight = 7.16)
cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
cb_model.fit(train, y_train,cat_features= categorical_features)
cb_model.cv_results_, cb_model.best_params_, cb_model.best_score_



**VOTING CLASSIFIER**

In [ ]:
x= train.copy()

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import StratifiedKFold,KFold
from lightgbm  import LGBMClassifier
from catboost import CatBoostClassifier

#rf = RandomForestClassifier(n_estimators=50)

#categorical_features = ['Gender', 'Driving_License', 'Previously_Insured','Region_Code', 'Vehicle_Age','agerange', 'Vehicle_Damage']

rf_1 = LGBMClassifier(learning_rate=0.02, boosting_type='gbdt', max_depth=6,num_leaves = 32,   objective='binary', 
                      random_state=100, n_estimators=3000 ,reg_alpha=2, reg_lambda=3, n_jobs=-1, 
                      is_unbalance= True,
                     # class_weights =weigh,
                      categorical_feature = categorical_features)

rf_2 = CatBoostClassifier(learning_rate=0.01,
                          #subsample=0.085, 
                          depth=6, verbose=400,
                          bootstrap_type="Bayesian",border_count = 50, colsample_bylevel = 0.9, l2_leaf_reg = 9, 
                          cat_features = categorical_features,iterations=3000,
                          #class_weights= weigh, 
                          eval_metric='AUC',loss_function= 'Logloss',
                          bagging_temperature =  1, scale_pos_weight = 7.16, thread_count=-1
                          #random_seed': 42
                         )

rf_3= XGBClassifier( learning_rate =0.01, n_estimators=5000, max_depth=4,
                    min_child_weight=4, gamma=0.45, subsample=0.8, colsample_bytree=0.8, reg_alpha= 0.01,
                    reg_lambda= 25,objective= 'binary:logistic', nthread=-1, scale_pos_weight=7.16, seed=27)


rf = VotingClassifier(estimators=[('CatBoost_Best', rf_2),('XGBoost_Best', rf_3),], voting='soft',weights= [6,5])


In [ ]:
from sklearn.model_selection import StratifiedKFold,KFold
kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=10)
acc = []

for fold,(t_id,v_id) in enumerate(kf.split(x,y)):
    tx = x.iloc[t_id]; ty = y.iloc[t_id]
    vx = x.iloc[v_id]; vy = y.iloc[v_id]
    rf.fit(tx,ty #weights = weights,
          )
           
    val_y = rf.predict_proba(vx)[:,1]
    auc_score = roc_auc_score(vy,val_y)
    acc.append(auc_score)
    print(f"fold {fold} accuracy {auc_score}")

print(f"Mean accuracy score {np.mean(auc_score)}")

In [ ]:
pred= rf.predict_proba(test)[:,1]


In [ ]:
pre= pd.Series(pred)
sub = pd.concat([test_case_id, pre], axis=1)
sub.rename({0: 'Response'},axis=1, inplace= True)
sub.head()

In [ ]:
sub.to_csv('latesub.csv', index=False)

In [ ]:
#With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train)
auc(clf, train, test,cat_features= categorical_features)



In [ ]:
#With Categorical features
clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, 
                            depth=10, iterations= 500, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train, cat_features= categorical_features)
auc(clf, train, test)

In [ ]:
from catboost import CatBoostClassifier
import numpy as np

model = CatBoostClassifier(loss_function='Logloss',scale_pos_weight=7.16)

grid = {'learning_rate': [0.001,0.01,0.1,1],
        #'iterations': [1000,2000],
        #'l2_leaf_reg': [1, 3, 5, 7, 9]
       }

grid_search_result = GridSearchCV(estimator=model, param_grid = grid, scoring= 'roc_auc')

grid_search_result.fit(train, y)

grid_search_result.cv_results_, grid_search_result.best_params_, grid_search_result.best_score_


In [ ]:
train.isnull().sum()

In [ ]:
clf = CatBoostClassifier()

cat_dims = [train.columns.get_loc(i) for i in categorical_features[:-1]] 
clf.fit(train, np.ravel(y), cat_features=categorical_features)
res = clf.predict(test)
print('error:',1-np.mean(res==np.ravel(test_label)))


In [ ]:
def catboost_param_tune(params,train_set,train_label,cat_dims=None,n_splits=3):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(ps.grid_search(['border_count']),
                      ps.grid_search(['ctr_border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms,train_set,train_label,cat_dims,n_splits)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        print(res,prms,'best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()


In [ ]:
def crossvaltest(params,train_set,train_label,cat_dims,n_splits=5):
    kf = KFold(n_splits=n_splits,shuffle=True) 
    res = []
    for train_index, test_index in kf.split(train_set):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    return np.mean(res)